In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window; 


### **Data Reading**

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@databrickproject1.dfs.core.windows.net/orders")

df.display()

In [0]:
df.printSchema()

In [0]:
df = df.withColumnRenamed('_rescued_data', 'rescued_data')

In [0]:
df = df.drop('rescued_data')
display(df)

In [0]:
from pyspark.sql.functions import to_timestamp, col

df = df.withColumn("order_date", to_timestamp(col('order_date')))
df.display()

In [0]:
from pyspark.sql.functions import year

df = df.withColumn("year", year(col("order_date")))
display(df)

In [0]:
df1 = df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1 = df1.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()
# df2 = df1.filter(df1.year == 2024)
# df2.display()

In [0]:
df1 = df1.withColumn("row_flag", row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()

### **Class - OOP**


In [0]:
class windows:
    # def __init__(self,df):
    #     self.df  = df

    def dense_rank(self,df):
        df_dense_rank =  df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_dense_rank
    def rank(self,df):
        df_rank = df.withcolumn("rank_flag",rank().over(window.partitionby("year").orderby(desc("total_amount"))))
        return df_rank
    def row_number(self,df):
        df_row_number = df.withcolumn("row_flag",row_number().over(window.partitionby("year").orderby(desc("total_amount"))))
        return df_row_number

In [0]:
new_df = df
display(new_df)

In [0]:
wobj = windows()
df_result = wobj.dense_rank(new_df)

display(df_result)

### **Data Writing**

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@databrickproject1.dfs.core.windows.net/orders")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.orders
using delta 
LOCATION "abfss://silver@databrickproject1.dfs.core.windows.net/orders"